In [91]:
import numpy as np

In [92]:
def g(p, beta):
    """
    Implements the Potts model tree recursion.
    
    Inputs: 
    p: a (d x q) matrix, giving the marginal probabilities for each of the children. 
    d here is the number of children and q the number of colors.
    
    beta: the temperature.
    
    
    Output: a row vector in R^q, giving the marginal probabilities of the parent.
    """
    numerator = np.prod(1 - (1-beta) * p, axis=0)
    denominator = np.sum(numerator)
    return numerator / denominator

def J(p, beta, Psi=lambda x: x*0+1):
    """
    Calculates the Jacobian of the tree recursion.
    
    Inputs:
    p: a (d x q) matrix, giving the marginal probabilities for each of the children.
    d here is the number of children and q the number of colors.

    beta: the temperature.
    
    Psi: (a python function) the derivative of the potential function. It's important 
    that this function is implemented in such a way that in can be applied elementwise
    to numpy arrays. By default the constant 1 function.
    
    Output: a (d x q x q) 3-tensor J, that represents the Jacobian of the tree recursion. 
    For each i \in [d], J_i is the Jacobian with respect to just the i'th child's marginal
    probabilities.
    """
    term1 = (1-beta) * ( np.outer(g(p, beta), g(p, beta)) - np.diag(g(p, beta)) )
    
    def to_apply(p_i):
        return np.diag(Psi(g(p,beta))) @ term1 @ np.diag(1/(1-(1-beta)*p_i)) @ np.diag(1/Psi(p_i))
    
    return np.apply_along_axis(to_apply, 1, p)

In [93]:
def Jacobian_test():
    """Numerical small example, with Psi = 1."""
    p = np.array([[1,0], [1,0]])
    betas = np.linspace(0.1, 2, 5)
    for b in betas:
        Jac = J(p, b)
        for i in range(len(Jac)):
            if not np.allclose(Jac[0], Jac[i]):
                print("Children jacobians not equal with equal marginals:")
                print(Jac[0])
                print(Jac[i])
                return False
        true_val = np.array([
            [-(1-b)*b/(1+b**2) + (1-b)*b**2*b/(1+b**2)**2, (1-b)*b**2 / (1+b**2)**2],
            [(1-b)*b / (1+b**2)**2, -(1-b)/(1+b**2) + (1-b)/(1+ b**2)**2]
        ])
        if not np.allclose(Jac[0], true_val):
            print("Jacobians J_i give the wrong value.")
            print("Obtained:")
            print(Jac[0])
            print("Truth:")
            print(true_val)
            return False
    return True

if not Jacobian_test():
    print("Test 1 failed.")
else:
    print("Test passed.")

Test passed.


In [90]:
# IMPORTANT NOTE: this matrix norm is **just the 2-norm**, and hence not exactly the same as what's used in Kuikui's
# colorings paper. I chose to use this as it's easier to implement. I'm not sure how important it is to stick to 
# Kuikui's norm here.
def norm_2(Jac):
    """Calculates the 2-norm (operator norm) of the Jacobian Jac.
    
    Input: a (d x q x q) tensor Jac, as outputted by the J function above.
    
    Output: The 2-norm of Jac, interpreted as a (q x dq) matrix.
    """
    return np.linalg.norm(np.concatenate(Jac, axis=1), 2)

Next we try testing out the colorings example, with Psi(x) = 1/(sqrt(x)(1-x)), q=9 and d=3. Note that this satisfies q >= d + 3sqrt(d).

# Issue found
We've hit an issue here. There are too many points to evaluate the Jacobian on. If we want to make progress, we need to algorithmically find the point p such that || J(p) || is maximized, and do so efficiently. It feels like this should be doable (?), but for now I'll leave it aside.

In [84]:
d = 3
q = 9

probability_vals = np.linspace(0, 1, 5)
sub_marginals = 

11.70820393249937

In [86]:
(5**9)**3

7450580596923828125